<a href="https://colab.research.google.com/github/ayu1729/DBAL/blob/master/ActiveLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier

# build function for the Keras' scikit-learn API
def create_keras_model():
    """
    This function compiles and returns a Keras model.
    Should be passed to KerasClassifier in the Keras scikit-learn API.
    """

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
pip install keras

In [ ]:
classifier=KerasClassifier(create_keras_model)

In [ ]:
import numpy as np
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
X_train=X_train.reshape(60000,28,28,1).astype("float32")/255
X_test=X_test.reshape(10000,28,28,1).astype("float32")/255
y_train = keras.utils.np_utils.to_categorical(y_train, 10)
y_test = keras.utils.np_utils.to_categorical(y_test, 10)
n_initial = 25000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)


(60000, 28, 28)


In [ ]:
print(y_test.shape)

(10000, 10)


In [ ]:
from modAL.models import ActiveLearner

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier)
    


In [ ]:
pip install modAL

In [ ]:
learner.fit(X_initial,y_initial)

782/782 [==============================] - 69s 87ms/step - loss: 0.5338 - accuracy: 0.8279


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


ActiveLearner(X_training=array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0....
              bootstrap_init=None,
 

In [ ]:
pip install modAl

In [ ]:
n_queries = 10
for idx in range(n_queries):
    print('Query no. %d' % (idx + 1))
    query_idx, query_instance = learner.query(X_pool, n_instances=2500, verbose=0)
    learner.teach(
        X=X_pool[query_idx], y=y_pool[query_idx], only_new=True,
        verbose=1
    )
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)

Query no. 1
79/79 [==============================] - 8s 85ms/step - loss: 1.6112 - accuracy: 0.4664
Query no. 2
79/79 [==============================] - 8s 85ms/step - loss: 0.5782 - accuracy: 0.8693
Query no. 3
79/79 [==============================] - 8s 86ms/step - loss: 0.9670 - accuracy: 0.7043
Query no. 4
79/79 [==============================] - 8s 86ms/step - loss: 1.0498 - accuracy: 0.6866
Query no. 5
79/79 [==============================] - 8s 86ms/step - loss: 0.9495 - accuracy: 0.7062
Query no. 6
79/79 [==============================] - 8s 85ms/step - loss: 1.0547 - accuracy: 0.6861
Query no. 7
79/79 [==============================] - 8s 87ms/step - loss: 1.0890 - accuracy: 0.6544
Query no. 8
79/79 [==============================] - 8s 87ms/step - loss: 1.0157 - accuracy: 0.6873
Query no. 9
79/79 [==============================] - 8s 87ms/step - loss: 1.1459 - accuracy: 0.6207
Query no. 10
79/79 [==============================] - 8s 86ms/step - loss: 0.9593 - accuracy: 0.7420

In [ ]:
yhat=learner.predict(X_test)
check_y=np.argmax(y_test, axis=1)
#print(check_y[1])
from sklearn.metrics import accuracy_score
accuracy_score(check_y,yhat)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9761